In [215]:
import tensorflow as tf
import pandas as pd
import datetime
import numpy as np
import os
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense, Dropout, Lambda
from keras.callbacks import Callback
import keras
from tensorflow.python import pywrap_tensorflow
import matplotlib.pyplot as plt
%matplotlib inline

## Preprocessing

In [218]:
'''
PARAMETERS TO CHANGE
'''
stock_symbol = 'bac'
train_model = True
test_model = True
learning_rate = 0.1
''''''

training_data_dir = './training_data/'
y_columns = ['ppo', 'rsi']
#assert(train_model ^ test_model)
model_dir = "./regularizer_models/" + stock_symbol + '/'

if train_model:
    training_data = pd.read_csv(training_data_dir + stock_symbol + '_training_data_smoothed.csv')
    unneeded_columns = ['window', 'Date_to_use_on']

    x_columns = list(training_data.columns)
    for to_remove in (y_columns + unneeded_columns):
        x_columns.remove(to_remove)

    train_x = training_data[x_columns].values
    train_y = training_data[y_columns].values

    model_dir = "./regularizer_models/" + stock_symbol + '/'

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        
if test_model:
    test_data = pd.read_csv(training_data_dir + stock_symbol + '_test_data.csv')
    
    unneeded_columns = ['Date_to_use_on']
    x_columns = list(test_data.columns)

    for to_remove in (unneeded_columns):
        x_columns.remove(to_remove)

    
    test_x = test_data[x_columns].values
    
    
    

In [217]:
'''data = pd.read_csv(training_data_dir + stock_symbol + '_training_data.csv').sort_values("window")
data.reset_index(drop=True, inplace=True)

for index, row in data.iterrows():
    if index > 0 and index < data.shape[0] - 1:
        prev_row = data.iloc[index-1]
        next_row = data.iloc[index+1]
        if row['ppo'] != prev_row['ppo'] and row['rsi'] != prev_row['rsi']:
            if prev_row['ppo'] == next_row['ppo'] and prev_row['rsi'] == next_row['rsi']:
                print(index)
                data['ppo'].loc[index] = prev_row['ppo']
                data['rsi'].loc[index] = prev_row['rsi']
                
data.to_csv(training_data_dir + stock_symbol + '_training_data_smoothed.csv', index=False)'''

306


c:\users\jing yang\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


566
1981
2025


## ANN Model

In [66]:
'''# Parameters

num_epochs = 30000
batch_size = 2075
display_epoch = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = len(x_columns) # Number of features
num_output = len(y_columns) # PPO, RSI parameters


def ANN(X):
    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_hidden_2, num_output]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([num_output]))
    }

    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)    
    
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    
    # Output fully connected layer with a neuron for each output
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    out = tf.square(out_layer)
    
    return out

X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_output])

output = ANN(X)

# Loss & Optimizer
loss_op = tf.losses.mean_squared_error(Y, output, weights=1)
train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_op)
# Accuracy
accuracy =  tf.abs(tf.reduce_mean(tf.round(output) - Y))'''

'# Parameters\n\nnum_epochs = 30000\nbatch_size = 2075\ndisplay_epoch = 100\n\n# Network Parameters\nn_hidden_1 = 256 # 1st layer number of neurons\nn_hidden_2 = 256 # 2nd layer number of neurons\nnum_input = len(x_columns) # Number of features\nnum_output = len(y_columns) # PPO, RSI parameters\n\n\ndef ANN(X):\n    # Store layers weight & bias\n    weights = {\n        \'h1\': tf.Variable(tf.random_normal([num_input, n_hidden_1])),\n        \'h2\': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),\n        \'out\': tf.Variable(tf.random_normal([n_hidden_2, num_output]))\n    }\n    biases = {\n        \'b1\': tf.Variable(tf.random_normal([n_hidden_1])),\n        \'b2\': tf.Variable(tf.random_normal([n_hidden_2])),\n        \'out\': tf.Variable(tf.random_normal([num_output]))\n    }\n\n    # Hidden fully connected layer with 256 neurons\n    layer_1 = tf.add(tf.matmul(X, weights[\'h1\']), biases[\'b1\'])\n    layer_1 = tf.nn.relu(layer_1)    \n    \n    # Hidden fully connected 

# LSTM Model

In [209]:
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='loss', value=20, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        #if current is None:
            #warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

            
def accuracy(y_true, y_pred):
    return tf.abs(tf.reduce_mean(tf.round(y_pred) - y_true))

window_size = 25
reshaped_train_x = np.reshape(train_x, train_x.shape + (1,))
model = Sequential()
model.add(LSTM(15, batch_input_shape=(2075, window_size, 1), dropout=0.5, recurrent_dropout=0.5, stateful=True)) # time step of 25, and input_dim of 1 per timestep
model.add(Dense(2,activation="linear"))
model.add(Lambda(lambda x: x ** 2))


callbacks = [
    EarlyStoppingByLossVal(monitor='loss', value=17, verbose=1),
]
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[accuracy])
model.fit(reshaped_train_x, train_y, batch_size = 2075, verbose=2, epochs=200, shuffle=True)#, callbacks=callbacks)

Epoch 1/200
 - 6s - loss: 30.4082 - accuracy: 3.5178
Epoch 2/200
 - 0s - loss: 30.4061 - accuracy: 3.5178
Epoch 3/200
 - 0s - loss: 30.4033 - accuracy: 3.5178
Epoch 4/200
 - 0s - loss: 30.3999 - accuracy: 3.5178
Epoch 5/200
 - 0s - loss: 30.3976 - accuracy: 3.5178
Epoch 6/200
 - 0s - loss: 30.3930 - accuracy: 3.5178
Epoch 7/200
 - 0s - loss: 30.3869 - accuracy: 3.5178
Epoch 8/200
 - 0s - loss: 30.3792 - accuracy: 3.5178
Epoch 9/200
 - 0s - loss: 30.3703 - accuracy: 3.5178
Epoch 10/200
 - 0s - loss: 30.3570 - accuracy: 3.5178
Epoch 11/200
 - 0s - loss: 30.3484 - accuracy: 3.5178
Epoch 12/200
 - 0s - loss: 30.3398 - accuracy: 3.5178
Epoch 13/200
 - 0s - loss: 30.3199 - accuracy: 3.5178
Epoch 14/200
 - 0s - loss: 30.2977 - accuracy: 3.5178
Epoch 15/200
 - 0s - loss: 30.2789 - accuracy: 3.5178
Epoch 16/200
 - 0s - loss: 30.2318 - accuracy: 3.5178
Epoch 17/200
 - 0s - loss: 30.1970 - accuracy: 3.5173
Epoch 18/200
 - 0s - loss: 30.1638 - accuracy: 3.5166
Epoch 19/200
 - 0s - loss: 30.0951 - 

Epoch 152/200
 - 0s - loss: 16.3801 - accuracy: 0.0643
Epoch 153/200
 - 0s - loss: 16.3683 - accuracy: 0.0602
Epoch 154/200
 - 0s - loss: 16.4088 - accuracy: 0.0745
Epoch 155/200
 - 0s - loss: 16.3158 - accuracy: 0.0629
Epoch 156/200
 - 0s - loss: 16.3542 - accuracy: 0.0684
Epoch 157/200
 - 0s - loss: 16.3404 - accuracy: 0.0723
Epoch 158/200
 - 0s - loss: 16.3124 - accuracy: 0.0687
Epoch 159/200
 - 0s - loss: 16.4659 - accuracy: 0.0708
Epoch 160/200
 - 0s - loss: 16.2526 - accuracy: 0.0586
Epoch 161/200
 - 0s - loss: 16.2439 - accuracy: 0.0812
Epoch 162/200
 - 0s - loss: 16.3613 - accuracy: 0.0520
Epoch 163/200
 - 0s - loss: 16.2843 - accuracy: 0.0653
Epoch 164/200
 - 0s - loss: 16.3507 - accuracy: 0.0639
Epoch 165/200
 - 0s - loss: 16.3474 - accuracy: 0.0634
Epoch 166/200
 - 0s - loss: 16.3284 - accuracy: 0.0692
Epoch 167/200
 - 0s - loss: 16.4157 - accuracy: 0.0680
Epoch 168/200
 - 0s - loss: 16.2562 - accuracy: 0.0651
Epoch 169/200
 - 0s - loss: 16.2636 - accuracy: 0.0513
Epoch 170/

In [210]:
reshaped_test_x = np.apply_along_axis(np.array_split, 1, test_x, indices_or_sections=window_size)
predicted = np.round(model.predict(reshaped_test_x))
result = pd.DataFrame(predicted, columns = y_columns)
result['Date_to_use_on'] = test_data['Date_to_use_on']
result = result[['Date_to_use_on'] + y_columns]
result.to_csv(training_data_dir+ stock_symbol + '_test_parameters.csv', index=False)

ValueError: In a stateful network, you should only pass inputs with a number of samples that can be divided by the batch size. Found: 2075 samples. Batch size: 32.

In [213]:
reshaped_test_x.shape

(2075, 25, 1)

In [6]:
'''if train_model:

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        # Run initializer
        sess.run(init)
        for epoch in range(num_epochs):
            
            shuffled_indexes = np.random.permutation(len(train_x))
    
            
            for i in shuffled_indexes[::batch_size]:
                batch_x = train_x[i:i+batch_size]
                batch_y = train_y[i:i+batch_size]

                sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})

            if epoch % display_epoch == 0 or epoch == 1:
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: train_x, Y: train_y})

                print("Epoch " + str(epoch) + ", Loss= " + \
                      "{:.4f}".format(loss) + ", Training Error= " + \
                      "{:.3f}".format(acc))

            if loss<20:
                break
        print(sess.run(output, feed_dict={X: train_x}))
        saver.save(sess, model_dir + stock_symbol + '_regularizer.ckpt')
        
        
        print('Model Saved')

      '''

Epoch 0, Loss= 264257929216.0000, Training Error= 489453.375
Epoch 1, Loss= 479811232.0000, Training Error= 17106.129
Epoch 100, Loss= 60911.2891, Training Error= 137.407
Epoch 200, Loss= 33908.1562, Training Error= 89.705
Epoch 300, Loss= 25045.1348, Training Error= 77.293
Epoch 400, Loss= 19477.2129, Training Error= 68.778
Epoch 500, Loss= 15974.9629, Training Error= 62.733
Epoch 600, Loss= 13465.0459, Training Error= 57.997
Epoch 700, Loss= 11610.8750, Training Error= 53.581
Epoch 800, Loss= 10056.4551, Training Error= 50.333
Epoch 900, Loss= 8817.3789, Training Error= 47.341
Epoch 1000, Loss= 7787.1768, Training Error= 44.355
Epoch 1100, Loss= 7028.2617, Training Error= 41.860
Epoch 1200, Loss= 6302.0996, Training Error= 39.665
Epoch 1300, Loss= 5750.4761, Training Error= 37.472
Epoch 1400, Loss= 5276.0107, Training Error= 35.894
Epoch 1500, Loss= 4782.9985, Training Error= 34.129
Epoch 1600, Loss= 4413.1641, Training Error= 33.068
Epoch 1700, Loss= 4043.2322, Training Error= 31.48

Epoch 16100, Loss= 42.2734, Training Error= 0.207
Epoch 16200, Loss= 37.9294, Training Error= 0.250
Epoch 16300, Loss= 37.2428, Training Error= 0.334
Epoch 16400, Loss= 33.6662, Training Error= 0.355
Epoch 16500, Loss= 34.9882, Training Error= 0.384
Epoch 16600, Loss= 31.3605, Training Error= 0.475
Epoch 16700, Loss= 28.9219, Training Error= 0.520
Epoch 16800, Loss= 28.5135, Training Error= 0.555
Epoch 16900, Loss= 27.5463, Training Error= 0.591
Epoch 17000, Loss= 27.0188, Training Error= 0.634
Epoch 17100, Loss= 27.1599, Training Error= 0.627
Epoch 17200, Loss= 26.0712, Training Error= 0.697
Epoch 17300, Loss= 25.9538, Training Error= 0.727
Epoch 17400, Loss= 25.3529, Training Error= 0.734
Epoch 17500, Loss= 24.8879, Training Error= 0.769
Epoch 17600, Loss= 24.8598, Training Error= 0.710
Epoch 17700, Loss= 24.3800, Training Error= 0.786
Epoch 17800, Loss= 24.3019, Training Error= 0.779
Epoch 17900, Loss= 23.4602, Training Error= 0.841
Epoch 18000, Loss= 23.2424, Training Error= 0.840


In [5]:
if test_model:
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        '''var_name_list = [v.name for v in tf.trainable_variables()]
        print(var_name_list)
        reader = pywrap_tensorflow.NewCheckpointReader(model_dir + stock_symbol + '_regularizer.ckpt')
        var_to_shape_map = reader.get_variable_to_shape_map()'''
        
        saver.restore(sess, model_dir + stock_symbol + '_regularizer.ckpt')
        print("Model restored.")
        predicted = tf.round(sess.run(output, feed_dict={X : test_x})).eval()
        predicted = np.array(predicted)
        
    result = pd.DataFrame(predicted, columns = y_columns)
    result['Date_to_use_on'] = test_data['Date_to_use_on']
    result = result[['Date_to_use_on'] + y_columns]
    result.to_csv(training_data_dir+ stock_symbol + '_test_parameters.csv', index=False)
    

INFO:tensorflow:Restoring parameters from ./regularizer_models/c/c_regularizer.ckpt


NotFoundError: Key Adam/beta_1 not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT64, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "c:\users\jing yang\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\jing yang\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\jing yang\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\jing yang\appdata\local\programs\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users

In [14]:
result

,Date_to_use_on,ppo,rsi
0,2009-11-20,4.0,1.0
1,2009-11-23,0.0,4.0
2,2009-11-24,0.0,1.0
3,2009-11-25,2.0,0.0
4,2009-11-27,0.0,0.0
5,2009-11-30,6.0,2.0
6,2009-12-01,2.0,0.0
7,2009-12-02,1.0,0.0
8,2009-12-03,1.0,4.0
9,2009-12-04,0.0,11.0
